## Notebook for reading in data and cleansing data, and dumping data with pickle

### Imports and settings

In [2]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split

RANDOM_STATE = 42
TRAINING_PORTITION = .8
GOOD_REVIEW_LIMIT = 3.5
CATEGORY_COLUMN_NAME = "category"

# en tiedä onko tuo varoitusten poistaminen hyvä idea, mutta joissain tilanteissa varoituksia alkoi tulemaan häiritsevän paljon
import warnings; warnings.filterwarnings(action='once')

In [14]:
# Kuvaajiin liittyvät asetukset ja funktiot
# https://www.machinelearningplus.com/plots/top-50-matplotlib-visualizations-the-master-plots-python/

# !pip install brewer2mpl
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import seaborn as sns
# large = 22; med = 16; small = 12
# params = {'axes.titlesize': large,
#           'legend.fontsize': med,
#           'figure.figsize': (16, 10),
#           'axes.labelsize': med,
#           'axes.titlesize': med,
#           'xtick.labelsize': med,
#           'ytick.labelsize': med,
#           'figure.titlesize': large}
# plt.rcParams.update(params)
# plt.style.use('seaborn-whitegrid')
# sns.set_style("white")
%matplotlib inline

# # Version
# print(mpl.__version__)  #> 3.0.0
# print(sns.__version__)  #> 0.9.0

### Datan lukeminen sisään

In [13]:
random_reviews = pd.read_csv(r'./Yelp_data/1000_random_reviews.csv', parse_dates=["date"])
# random_reviews.dropna(inplace=True) # ei tarvitty, kun ei ollut
bow1000 = pd.read_csv(r'./Yelp_data/bow1000.csv') 
# joined_df= pd.concat([random_reviews, bow1000], axis=1)
# joined_df.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id,able,...,writing,wrong,year,years,yelp,yes,yet,young,yum,yummy
0,KW5edXlPGeZ0cx7MiJyamw,0,2017-03-07 02:03:24,0,nkvJghZnOkZy4HAo87ouSA,5,"A stellar cafe!!! Very classy, warm & inviting...",0,PH2Ly-LUDc5i-eW_97MlNw,0,...,0,0,0,0,0,0,0,0,0,0
1,oxwGyA17NL6c5t1Etg5WgQ,0,2017-04-13 04:03:48,0,7KrKzKgUdflicfUs_bWNUQ,1,"Can you say ""Bait and Switch?"" A couple of da...",1,EnSUGj7a7FocW3blBj_hEw,0,...,0,0,0,0,0,0,0,0,0,0
2,bKxKPGmkwH-Q-GSbt7chNA,1,2013-08-11 20:05:49,1,f54iEgnNZEE-JPcdVuNXSQ,2,"When first walking in to Stingray Sushi, a low...",3,aY4FqMsNXBirS72lM_CdwQ,0,...,0,0,0,0,0,0,0,0,0,0
3,q5viyMO6EtJMmmarb4YmIA,0,2015-01-09 23:57:08,0,AtKH9dzGHIKoyAhOLcVUIw,4,A very beautiful restaurant in the Hardrock ca...,2,CTaZ3ebFGQMHaLZhnO2EuA,0,...,0,0,0,0,0,0,0,0,0,0
4,BS7PU7REvJZ7hVSHA6z8wQ,0,2017-09-03 22:40:45,0,C5vaxaN3IZvmBpvcv0uYXw,2,I've been here twice. The first time was a pre...,1,Kya8CWymuq437rxhHyiM2g,0,...,0,0,0,1,0,0,0,0,0,0


In [12]:
random_reviews.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,KW5edXlPGeZ0cx7MiJyamw,0,2017-03-07 02:03:24,0,nkvJghZnOkZy4HAo87ouSA,5,"A stellar cafe!!! Very classy, warm & inviting...",0,PH2Ly-LUDc5i-eW_97MlNw
1,oxwGyA17NL6c5t1Etg5WgQ,0,2017-04-13 04:03:48,0,7KrKzKgUdflicfUs_bWNUQ,1,"Can you say ""Bait and Switch?"" A couple of da...",1,EnSUGj7a7FocW3blBj_hEw
2,bKxKPGmkwH-Q-GSbt7chNA,1,2013-08-11 20:05:49,1,f54iEgnNZEE-JPcdVuNXSQ,2,"When first walking in to Stingray Sushi, a low...",3,aY4FqMsNXBirS72lM_CdwQ
3,q5viyMO6EtJMmmarb4YmIA,0,2015-01-09 23:57:08,0,AtKH9dzGHIKoyAhOLcVUIw,4,A very beautiful restaurant in the Hardrock ca...,2,CTaZ3ebFGQMHaLZhnO2EuA
4,BS7PU7REvJZ7hVSHA6z8wQ,0,2017-09-03 22:40:45,0,C5vaxaN3IZvmBpvcv0uYXw,2,I've been here twice. The first time was a pre...,1,Kya8CWymuq437rxhHyiM2g


In [5]:
print("Data import valmis")
print("-----------------------")
print("käytössä muuttujat:")
print("- random_reviews")
print("- bow1000")

Data import valmis
-----------------------
käytössä muuttujat:
- random_reviews
- bow1000


In [6]:
def train_test_split_as_requested(X, y, portition=TRAINING_PORTITION):
  print("palauttaa X_train, X_test, y_train, y_test")
  return train_test_split(X, y, test_size=1-portition, random_state=None, shuffle=False)

def get_training_portition_of_the_data(dataframe=random_reviews):
  train_size = int(len(dataframe) * TRAINING_PORTITION)
  print("palautetaan {} riviä yhteensä {} rivistä (osuus {})".format(train_size, len(dataframe), TRAINING_PORTITION))
  return dataframe[:train_size]

print("Käytössä funktio:")
print("- train_test_split_as_requested(X, y, portition)   joka palauttta testidatan kuten ohjeissa annettu, muodossa X_train, X_test, y_train, y_test")
print("- get_training_portition_of_the_data(dataframe=random_reviews)  joka palauttaa vakiomäärän dataa analyysejä varten")

Käytössä funktio:
- train_test_split_as_requested(X, y, portition)   joka palauttta testidatan kuten ohjeissa annettu, muodossa X_train, X_test, y_train, y_test
- get_training_portition_of_the_data(dataframe=random_reviews)  joka palauttaa vakiomäärän dataa analyysejä varten


## Lisätään good-bad -sarake
Categorize the data in 1000 random reviews.csv into two different classes: 

In [7]:
# poistetaan ensin vanha, jos on jo olemassa
if CATEGORY_COLUMN_NAME in random_reviews.columns:
  random_reviews.drop([CATEGORY_COLUMN_NAME], axis=1)
def label_category (row):
   if row['stars'] >= GOOD_REVIEW_LIMIT : #3.5
      return 1 #'good'
   return 0 #'poor'
random_reviews[CATEGORY_COLUMN_NAME] = random_reviews.apply (lambda row: label_category(row), axis=1)